In [1]:
import os
os.chdir('../')
%pwd

'e:\\Deep Learning\\pytorch\\facial_emotion_prediction'

In [2]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen= True)
class ModelPreparationTrainingConfig:
    root_dir: Path
    model_dir: Path
    train_dir: Path
    val_dir: Path
    history_dir: Path
    learning_rate: float
    classes: int
    epochs: int
    weight_decay: float
    input_image_size: list
    epsilon: float
    momentum: float
    decay_rate: float
    batch_size: int

In [3]:
from src.Facial_emotion_classification.constants import *
from src.Facial_emotion_classification.utils.common import read_yaml, create_directories

In [4]:
class ConfigureationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_prep_train_config(self) -> ModelPreparationTrainingConfig:
        config = self.config.model_preparation_training
        create_directories([config.root_dir])

        model_prep_train_config = ModelPreparationTrainingConfig(
            root_dir=Path(config.root_dir),
            model_dir=Path(config.model_dir),
            train_dir=Path(config.train_dir),
            val_dir=Path(config.val_dir),
            history_dir=Path(config.history_dir),
            classes=int(self.params.classes),
            learning_rate=float(self.params.learning_rate),
            epochs=int(self.params.epochs),
            weight_decay=float(self.params.weight_decay),
            input_image_size=[int(x) for x in self.params.input_image_size],
            epsilon=float(self.params.epsilon),
            momentum = float(self.params.momentum),
            decay_rate = float(self.params.decay_rate),
            batch_size=int(self.params.batch_size)
        )
        return model_prep_train_config

In [5]:

import torch
from torchsummary import summary
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR
import json
from src.Facial_emotion_classification import logger

class ModelPreparation:
    def __init__(self, config):
        self.config = config

    def model(self):
        cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),  # Changed to 3 input channels
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(64 * 12 * 12, 512),
            nn.ReLU(),
            nn.Linear(512, self.config.classes)
        )
        return cnn

    def image_processing(self):
        resize_size = self.config.input_image_size[-2:]

        transformer = transforms.Compose([
            transforms.Resize(resize_size),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
            transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Changed normalization to 3 channels
        ])

        train_loader = DataLoader(
            torchvision.datasets.ImageFolder(self.config.train_dir, transform=transformer),
            batch_size=self.config.batch_size, shuffle=True
        )
        val_loader = DataLoader(
            torchvision.datasets.ImageFolder(self.config.val_dir, transform=transformer),
            batch_size=self.config.batch_size, shuffle=True
        )

        train_count = len(train_loader.dataset)
        val_count = len(val_loader.dataset)

        return train_loader, val_loader, train_count, val_count

    def model_compilation(self, model):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)
        optimizer = optim.Adam(model.parameters(), lr=self.config.learning_rate, weight_decay=self.config.weight_decay)
        scheduler = ExponentialLR(optimizer, gamma=self.config.decay_rate)
        criterion = nn.CrossEntropyLoss()
        return model, optimizer, scheduler, criterion

    def train_model(self, model, optimizer, scheduler, criterion, train_loader, val_loader, train_count, val_count):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.train()

        metrics = {
            "train_loss": [],
            "train_accuracy": [],
            "val_loss": [],
            "val_accuracy": []
        }

        for epoch in range(self.config.epochs):
            train_loss = 0.0
            train_accuracy = 0
            val_loss = 0.0
            val_accuracy = 0

            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * inputs.size(0)
                _, prediction = torch.max(outputs.data, 1)
                train_accuracy += int(torch.sum(prediction == labels.data))

            train_accuracy = train_accuracy / train_count
            train_loss = train_loss / train_count

            model.eval()
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item() * inputs.size(0)
                    _, prediction = torch.max(outputs.data, 1)
                    val_accuracy += int(torch.sum(prediction == labels.data))

            val_accuracy = val_accuracy / val_count
            val_loss = val_loss / val_count

            print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
                  f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

            metrics["train_loss"].append(train_loss)
            metrics["train_accuracy"].append(train_accuracy)
            metrics["val_loss"].append(val_loss)
            metrics["val_accuracy"].append(val_accuracy)

        with open(self.config.history_dir, 'w') as f:
            json.dump(metrics, f, indent=4)

        logger.info("------------------Training And Evaluation Ended -------------------")
        return model

    def print_model_summary(self, model, input_size):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        summary(model, input_size, device=str(device))
        
    def save_model(self, model):
        model_path = self.config.model_dir
        torch.save(model.state_dict(), model_path)
        print(f'Model saved to {model_path}')


In [6]:
try:
    config = ConfigureationManager()
    model_preparation_training_config = config.get_model_prep_train_config()
    model_preparation = ModelPreparation(config=model_preparation_training_config)
    train_loader, val_loader, train_count, val_count = model_preparation.image_processing()
    cnn = model_preparation.model()
    model, optimizer, scheduler, criterion = model_preparation.model_compilation(cnn)
    model_preparation.print_model_summary(cnn, (3, 48, 48))
    model = model_preparation.train_model(model, optimizer, scheduler, criterion, train_loader, val_loader, train_count, val_count)
    model_preparation.save_model(model)
except Exception as e:
    raise e

[2024-08-01 20:58:41,136: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-01 20:58:41,142: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-01 20:58:41,142: INFO: common: created directory at: artifacts]
[2024-08-01 20:58:41,142: INFO: common: created directory at: artifacts/training]
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 48, 48]             896
              ReLU-2           [-1, 32, 48, 48]               0
         MaxPool2d-3           [-1, 32, 24, 24]               0
            Conv2d-4           [-1, 64, 24, 24]          18,496
              ReLU-5           [-1, 64, 24, 24]               0
         MaxPool2d-6           [-1, 64, 12, 12]               0
           Flatten-7                 [-1, 9216]               0
            Linear-8                  [-1, 512]       4,719,104
              ReLU-9         

KeyboardInterrupt: 